
# Co-folding with Boltz

*Disclaimer: This topic is a very active area of research and thus prone to quick changes. The Notebook was last edited on 30.06.2025*

---
### In this lesson you'll learn:

- how to predict a protein structure from the amino acid sequence.
- how to align two structures in python and calculate the RMSD in between.
- how to predict the structure of a protein-ligand complex from sequence and SMILES.
- about current limitations of co-folding methods.

---

This notebook is about protein folding, especially *co-folding*, i.e. not just the prediciton of the 3D protein structure from the sequence but also the placement of a ligand in the correct conformation and binding pocket.  

While protein-folding was largly solved by [Alphafold2](https://doi.org/10.1038/s41586-021-03819-2), newer models try to solve the structure of protein-ligand complexes from the sequence and a SMILES. Finding the binding conformation is typically done using docking software such as [gnina](https://github.com/gnina/gnina), [GOLD](https://www.ccdc.cam.ac.uk/solutions/software/gold/) or [Glide](https://www.schrodinger.com/platform/products/glide/). 

Google's [Alphafold3](https://github.com/google-deepmind/alphafold3) is still one of the top performing models for co-folding, but in this notebook we'll be using [Boltz](https://github.com/jwohlwend/boltz), specifically Boltz2. As of writing, this is the newest co-folding model and even offers binding strenght estimation, which we'll  have a look at at the end of the notebook. 

---

## Installation, Google Colab and the Command Line

**Optional Boltz Install**: To complete this notebook, it is not necessary to install Boltz, but we give the option to do so. Another great resource to try out Boltz in Google Colab is [this Notebook](https://colab.research.google.com/github/kimjc95/computational-chemistry/blob/main/Boltz_on_Colab.ipynb).

**Local install**: This Notebook is created to be run with Google Colab and the following cells will install Boltz and its dependencies. If you want to run Boltz locally (recommended if you have a strong GPU) please follow the instructions [here]() and install on linux or in WSL. Please then just execute the escaped linux commands (the commands starting with `!`, e.g. `!boltz predict example_file.fasta`). Note that you might need to resolve the paths (where the files are stored).

**Google Colab**: Make sure you are connected to a runtime with GPU support (go to the upper right corner, open the drop-down menu, select "change runtime type" and make sure a Hardware Accelerator other than CPU is selected).

**Command Line**: As Boltz is a command line program, we will be using a few linux commands instead of typical python code. You can recognize these commands by the `!` which is used to tell the python interpreter to pass this  command to the underlying operating system.

---

In [ ]:
# Install py3Dmol for viewing
!pip install py3Dmol

## (Optional) Run an actual Boltz Prediciton

In [ ]:
# Installation (for Google Colab only)
# this code was adapted from [Joo-Chan Kim](https://zenodo.org/records/14881401)

import os
import subprocess

print('Installing dependencies (estimate: 2min) ... ', end='')
dependencies = "torch torchvision torchaudio numpy hydra-core pytorch-lightning "
dependencies += "rdkit dm-tree requests pandas types-requests einops einx fairscale "
dependencies += "mashumaro modelcif wandb click pyyaml biopython scipy numba gemmi "
dependencies += "scikit-learn chembl_structure_pipeline "
dependencies += "cuequivariance_ops_cu12 cuequivariance_ops_torch_cu12 cuequivariance_torch"

precision = '32-true'

subprocess.run("pip install ipywidgets torch torchvision torchaudio", shell=True)
subprocess.run("git clone https://github.com/jwohlwend/boltz.git", shell=True)
subprocess.run(f"sed -i 's/bf16-mixed/{precision}/g' /content/boltz/src/boltz/main.py", shell=True)
subprocess.run(f"pip install {dependencies}", shell=True)
subprocess.run("cd boltz; pip install --no-deps -e .", shell=True)

print('done.')

Now we'll create a very simple fasta file using some linux commands:

In [ ]:
# `echo` is the linux way of using `print()` and with the `>` we write the ouput to a file
!echo -e ">A|protein|empty\nAAAA\n" > peptide.fasta

In [ ]:
# we can have a look at the file with `cat`
!cat peptide.fasta

Now we can use `boltz predict peptide.fasta` to predict the structure of the small peptide.
Note that we used the keyword `empty` in our `.fasta` file. This will lead to much worse performance, because we don't do any sequence alignment.
Normally, we would have to add the `--use_msa_server` keyword to use an external server for the sequence alignment (or a supply our own .a3m file).

In [ ]:
!boltz predict peptide.fasta

The output is saved in the folder `boltz_results_peptide` and we can use Google Colabs file browser (the folder symbol on the left hotbar). The actual predicted 3D structure can be visualized using py3Dmol.

In [ ]:
import py3Dmol 
fasta_name = 'peptide'

with open(f"boltz_results_{fasta_name}/predictions/{fasta_name}/{fasta_name}_model_0.cif") as ifile:
    system = "".join([x for x in ifile])

view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(system)
view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})
view.zoomTo()
view.show()

There are many [more settings](https://github.com/jwohlwend/boltz/blob/main/docs/prediction.md) available to run Boltz with, this section should just give you a point to get started with co-folding.

## Comparing crystal structures and predicted structures

While it is possible to run the following examples in Google Colab, the freely available GPUs are not that fast and might run out of memory for larger sequences. We thus ran the predictions locally, so you don't have to wait for the models. We do show you the code to generate the structures, feel free to run them (in Google Colab if you have the time, or locally). Other options would be to generate structures using [Alphafold-Server](https://alphafoldserver.com/welcome), or using a local install of [Alphafold3](https://github.com/google-deepmind/alphafold3). Note, that for these options you will have to look in the documentation of these methods on how to prepare the files for these models.

We'll have a look at the **AKT1 kinase**. Kinases are abundand in the PDB, and (co-)folding models should be able to easily produce a correct structure.

First, lets just have a look at the pdb entry, we'll start with the entry [`3MVH`](https://www.rcsb.org/structure/3MVH) which already has an orthosteric inhibitor bound:

In [ ]:
import py3Dmol

view = py3Dmol.view(width=800, height=600, query="3mvh")
view.setStyle({'cartoon':{'color':'spectrum'}})

view.show()


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

To predict the structure, we can get the protein sequence. While we could use the one from the PDB structure above, the sequence might be truncated or mutated (e.g. for easier crystallization), so we'll take the sequence from the [UniProt](https://www.uniprot.org/) database (ID: P31749)

```
>sp|P31749|AKT1_HUMAN RAC-alpha serine/threonine-protein kinase OS=Homo sapiens OX=9606 GN=AKT1 PE=1 SV=2
MSDVAIVKEGWLHKRGEYIKTWRPRYFLLKNDGTFIGYKERPQDVDQREAPLNNFSVAQC
QLMKTERPRPNTFIIRCLQWTTVIERTFHVETPEEREEWTTAIQTVADGLKKQEEEEMDF
RSGSPSDNSGAEEMEVSLAKPKHRVTMNEFEYLKLLGKGTFGKVILVKEKATGRYYAMKI
LKKEVIVAKDEVAHTLTENRVLQNSRHPFLTALKYSFQTHDRLCFVMEYANGGELFFHLS
RERVFSEDRARFYGAEIVSALDYLHSEKNVVYRDLKLENLMLDKDGHIKITDFGLCKEGI
KDGATMKTFCGTPEYLAPEVLEDNDYGRAVDWWGLGVVMYEMMCGRLPFYNQDHEKLFEL
ILMEEIRFPRTLGPEAKSLLSGLLKKDPKQRLGGGSEDAKEIMQHRFFAGIVWQHVYEKK
LSPPFKPQVTSETDTRYFDEEFTAQMITITPPDQDDSMECVDSERRPHFPQFSYSASGTA
```
(from https://rest.uniprot.org/uniprotkb/P31749.fasta)

We'll quickly  modify the fasta so, as boltz expects the sequence type and a path to an `a3m` file (which we'll leave empty):


In [30]:
with open('p31749.fasta', 'w') as f:
    f.write(
"""
>A|protein|
MSDVAIVKEGWLHKRGEYIKTWRPRYFLLKNDGTFIGYKERPQDVDQREAPLNNFSVAQC
QLMKTERPRPNTFIIRCLQWTTVIERTFHVETPEEREEWTTAIQTVADGLKKQEEEEMDF
RSGSPSDNSGAEEMEVSLAKPKHRVTMNEFEYLKLLGKGTFGKVILVKEKATGRYYAMKI
LKKEVIVAKDEVAHTLTENRVLQNSRHPFLTALKYSFQTHDRLCFVMEYANGGELFFHLS
RERVFSEDRARFYGAEIVSALDYLHSEKNVVYRDLKLENLMLDKDGHIKITDFGLCKEGI
KDGATMKTFCGTPEYLAPEVLEDNDYGRAVDWWGLGVVMYEMMCGRLPFYNQDHEKLFEL
ILMEEIRFPRTLGPEAKSLLSGLLKKDPKQRLGGGSEDAKEIMQHRFFAGIVWQHVYEKK
LSPPFKPQVTSETDTRYFDEEFTAQMITITPPDQDDSMECVDSERRPHFPQFSYSASGTA
"""
    )


With this, we can run a Boltz prediction. If you want to run the prediction yourself, uncomment the first line and comment out the rest of the next cell.

In [ ]:
# !boltz predict p31749.fasta --use_msa_server
!wget -O boltz_results_p31749.zip https://uni-muenster.sciebo.de/s/qyFes2eQQApmroC/download
!unzip 
!rm boltz_results_p31749.zip

We can have a look at the predicted fold and compare it to the crystal structure:

In [70]:
name = 'p31749'

with open(f"boltz_results_{name}/predictions/{name}/{name}_model_0.cif") as ifile:
    system = "".join([x for x in ifile])

view = py3Dmol.view(width=800, height=600, query='pdb:3mvh', linked=True)
view.addModelsAsFrames(system)
view.setStyle({'model': -1}, {"cartoon": {'color': 'red'}}) # our prediction
view.setStyle({'model': -2}, {"cartoon": {'color': 'green'}}) # PDB:3mvh
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

The predicted structure (red) does look like a sensible protein, with some alpha helices and beta sheets. However, since the two structures aren't aligned, we can't really tell how good the prediction is. So that is what we'll tackle next.

### Aligning Structures in Python




<details>
<summary><strong>How to hide part of a cell:</strong></summary>

hidden details

</details>

In [ ]:
import py3Dmol 

with open("boltz_results_ligand/predictions/ligand/ligand_model_0.cif") as ifile:
# s.listdir('boltz_results_ligand/predictions/ligand/ligand_model0.cif')
    system = "".join([x for x in ifile])

view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(system)
view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})
view.zoomTo()
view.show()

In [26]:
len('MSDVAIVKEGWLHKRGEYIKTWRPRYFLLKNDGTFIGYKERPQDVDQREAPLNNFSVAQCQLMKTERPRPNTFIIRCLQWTTVIERTFHVETPEEREEWTTAIQTVADGLKKQEEEEMDFRSGSPSDNSGAEEMEVSLAKPKHRVTMNEFEYLKLLGKGTFGKVILVKEKATGRYYAMKILKKEVIVAKDEVAHTLTENRVLQNSRHPFLTALKYSFQTHDRLCFVMEYANGGELFFHLSRERVFSEDRARFYGAEIVSALDYLHSEKNVVYRDLKLENLMLDKDGHIKITDFGLCKEGIKDGATMKTFCGTPEYLAPEVLEDNDYGRAVDWWGLGVVMYEMMCGRLPFYNQDHEKLFELILMEEIRFPRTLGPEAKSLLSGLLKKDPKQRLGGGSEDAKEIMQHRFFAGIVWQHVYEKKLSPPFKPQVTSETDTRYFDEEFTAQMITITPPDQDDSMECVDSERRPHFPQFSYSASGTA')

480

### References

- Xie J, Wang S, Xu Y, Deng M, Lai L. Uncovering the Dominant Motion Modes of Allosteric Regulation Improves Allosteric Site Prediction. J Chem Inf Model 2022;62:187–95.
- Olanders G, Testa G, Tibo A, Nittinger E, Tyrchan C. Challenge for Deep Learning: Protein Structure Prediction of Ligand-Induced Conformational Changes at Allosteric and Orthosteric Sites. J Chem Inf Model 2024.